In [1]:
from nba_api.stats.static import players
player_dict = players.get_players()

In [2]:
type(player_dict)

list

In [3]:
tatum = [player for player in player_dict if player['full_name'] == 'Jayson Tatum'][0]
tatum_id = tatum['id']

In [4]:
# First we import the endpoint
# We will be using pandas dataframes to manipulate the data
from nba_api.stats.endpoints import playergamelog
import pandas as pd 


In [ ]:
import requests
# set timeout to 50 seconds
requests.get('https://www.google.com', timeout=50)

In [8]:
from nba_api.stats.endpoints import playercareerstats


headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'Keep Alive',
    'Keep-Alive': 'timeout = 60',
}


career = playercareerstats.PlayerCareerStats(player_id=tatum_id, headers=headers) 


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [8]:
import requests
# set timeout to 50 seconds
tatum = requests.get("https://www.espn.co.uk/nba/player/gamelog/_/id/4065648/jayson-tatum", timeout=50)

In [20]:
tatum.json()

JSONDecodeError: Expecting value: line 2 column 9 (char 9)

In [56]:
# scrape data on our own w/ read_html
# will return a bunch of different dfs in a list
# for now (since we're in november), we only want 0 and 1
# however, in subsequent months, we will want every month
# we'll get each dataframe in a loop and stop once it starts giving us season averages (at bottom of webpage)

# full dataframe for our amalgamated data from webpage
data = pd.DataFrame()

# df of all tables on webpage
df = pd.read_html("https://www.espn.co.uk/nba/player/gamelog/_/id/4065648/jayson-tatum")


for record in range(len(df)): # each table pulled from website

    # check if we see 'Regular Season Stats' as first col - indicates that we've reached the tables we don't want
    if df[record].columns[0] == 'Regular Season Stats':
        print(f"{record} tables found.")
        break

    # if conditional isn't hit, clean df and append to data
    df[record].drop(df[record].tail(1).index, inplace=True) # drop last row - totals for the month
    data = data.append(df)
    

2 tables found.


/tmp/ipykernel_13148/1147837088.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df)


In [57]:
data

,Date,OPP,Result,MIN,FG,FG%,3PT,3P%,FT,FT%,...,PF,TO,PTS,Regular Season Stats,Team,W,L,PCT,GB,STRK
0,Sat 11/19,@NO,W117-109,39.0,6-18,33.3,2-10,20.0,5-6,83.3,...,1.0,3.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu 11/17,@ATL,W126-101,33.0,6-19,31.6,3-10,30.0,4-4,100.0,...,1.0,2.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tue 11/15,vsOKC,W126-122,38.0,9-23,39.1,1-9,11.1,8-10,80.0,...,2.0,1.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sun 11/13,@DET,W117-108,41.0,14-28,50.0,7-14,50.0,8-10,80.0,...,2.0,3.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sat 11/12,vsDEN,W131-112,37.0,11-21,52.4,2-7,28.6,10-11,90.9,...,4.0,1.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Thu 11/10,vsDET,W128-112,32.0,10-20,50.0,5-11,45.5,6-7,85.7,...,4.0,1.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tue 11/8,@MEM,W109-106,40.0,12-25,48.0,3-11,27.3,12-16,75.0,...,2.0,2.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sat 11/5,@NY,W133-118,37.0,8-17,47.1,6-13,46.2,4-4,100.0,...,3.0,3.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Fri 11/4,vsCHI,W123-119,40.0,8-23,34.8,3-10,30.0,17-20,85.0,...,4.0,1.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Wed 11/2,@CLE,L114-113 OT,43.0,8-21,38.1,2-9,22.2,8-8,100.0,...,0.0,6.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
